# 1. Load dataset
#### In this assignment, you are expected to build an ensemble of different models and train it on cover type dataset.
#### You will need to read the data from the file (cover.csv). It contains 581012 samples and 54 attributes for each sample.

In [2]:
import pandas as pd
df=pd.read_csv('cover.csv')
df

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Cover_Type
0,2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
1,2590.0,56.0,2.0,212.0,-6.0,390.0,220.0,235.0,151.0,6225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
2,2804.0,139.0,9.0,268.0,65.0,3180.0,234.0,238.0,135.0,6121.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,2785.0,155.0,18.0,242.0,118.0,3090.0,238.0,238.0,122.0,6211.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,2595.0,45.0,2.0,153.0,-1.0,391.0,220.0,234.0,150.0,6172.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396.0,153.0,20.0,85.0,17.0,108.0,240.0,237.0,118.0,837.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
581008,2391.0,152.0,19.0,67.0,12.0,95.0,240.0,237.0,119.0,845.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
581009,2386.0,159.0,17.0,60.0,7.0,90.0,236.0,241.0,130.0,854.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
581010,2384.0,170.0,15.0,60.0,5.0,90.0,230.0,245.0,143.0,864.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


# 2. Prepare dataset
#### Split the data into train, validation, and test sets.

In [16]:
y=df['Cover_Type']
X=df.iloc[:,:-1]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=0.2, shuffle = True, random_state = 8)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
    test_size=0.25, random_state= 8)

# 3. Modeling
#### Train different classifiers on the data. You can train RandomForestClassifier, ExtraTreesClassifier, LinearSVC, SGDClassifier, MLPClassifier, etc. Evaluate their performance using validation set.

In [26]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_jobs=-1)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_val)
print(accuracy_score(y_val, y_pred))

0.9489686152681084


In [20]:
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier()
etc.fit(X_train,y_train)
yetc_pred=etc.predict(X_val)
print(accuracy_score(y_val, yetc_pred))

0.9484780943693364


In [28]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
svc=make_pipeline(StandardScaler(),LinearSVC())
svc.fit(X_train,y_train)
ysvc_pred=svc.predict(X_val)
print(accuracy_score(y_val, ysvc_pred))

TypeError: __init__() got an unexpected keyword argument 'n_jobs'

In [29]:
from sklearn.linear_model import SGDClassifier
sgd=make_pipeline(StandardScaler(),SGDClassifier())
sgd.fit(X_train,y_train)
ysgd_pred=sgd.predict(X_val)
print(accuracy_score(y_val, ysgd_pred))

0.7139230484583015


# 4. Ensembling
#### Create a hard and soft voting classifier using the models you have trained. You can use VotingClassifier. Check its performance on the validatioin set. Check if any of the models hurts the performance of the ensemble.

In [25]:
from sklearn.ensemble import VotingClassifier
ensemble_clf_hard = VotingClassifier(estimators=[
         ('rfs', clf), ('etc',etc )], voting='hard')

ensemble_clf_hard.fit(X_train, y_train)
ensemble_clf_soft = VotingClassifier(estimators=[
         ('rfs', clf), ('etc',etc )],voting='soft')
ensemble_clf_soft.fit(X_train, y_train)
yhard_pred=ensemble_clf_hard.predict(X_val)
ysoft_pred=ensemble_clf_soft.predict(X_val)
print(accuracy_score(y_val, yhard_pred),"Hard")
print(accuracy_score(y_val, ysoft_pred),"Soft")

0.9481510804368217 Hard
0.9499582626954554 Soft


#### Make predictions on the validation set using trained models and create a new training set out of those predictions: each training example will now have predictions of all classifiers as features. Train a new classifier on this new training set. Compare the performances.

In [34]:
X_val=X_val.assign(rfc=y_pred)
len(list(X_val.columns))

55

In [35]:
X_val=X_val.assign(etc=yetc_pred)
len(list(X_val.columns))

56

In [36]:
X_train1, X_val1, y_train1, y_val1 = train_test_split(X_val, y_val, 
    test_size=0.25, random_state= 8)

In [37]:
clf1 = RandomForestClassifier(n_jobs=-1)
clf1.fit(X_train1, y_train1)
y_predd=clf1.predict(X_val1)
print(accuracy_score(y_val1, y_predd))

0.9524284878317442
